<a href="https://colab.research.google.com/github/opsabarsec/NLP--film-genres-from-synopsis/blob/main/radix_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic assignment of genres from movie synopsis using supervised machine learning

## 1. Import libraries and load data

In [ ]:
#packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# NLP libraries

from textblob import TextBlob, Word
import nltk
import re
import csv
from sklearn.preprocessing import MultiLabelBinarizer

# Deep learning libraries

from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, Input, LSTM, GlobalMaxPool1D, Dropout
from keras.preprocessing.text import Tokenizer

In [ ]:
# load data
train= pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

## 2. DATA PREPARATION 

In [ ]:
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/marco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# function for text cleaning 
def preprocess_text(text):
    text = text.lower() # lowercase
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text) #line breaks
    #text = re.sub(r"\'\xa0", " ", text) # xa0 Unicode representing spaces
    #text = re.sub('\s+', ' ', text) # one or more whitespace characters
    text = text.strip(' ') # spaces
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    #lemmatize and remove stopwords
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text) 
        
    return text

train['clean_plot'] = train['synopsis'].apply(lambda x: preprocess_text(x))
test['clean_plot'] = test['synopsis'].apply(lambda x: preprocess_text(x))

In [ ]:
def lemma(text): # Lemmatization of cleaned body
        sent = TextBlob(text)
        tag_dict = {"J": 'a', 
                    "N": 'n', 
                    "V": 'v', 
                    "R": 'r'}
        words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
        lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
        seperator=' '
        lemma = seperator.join(lemmatized_list) 
        return lemma

In [ ]:
train['lemma'] = train['clean_plot'].apply(lambda x: lemma(x))
test['lemma'] = test['clean_plot'].apply(lambda x: lemma(x))

## 3. Variables preparation 

In [ ]:

X = train['lemma']

X_test = test['lemma']    

### 3.1 Target variable one hot encoding

In [ ]:
#apply the onehot transformation for the genres vector
y = train['genres']
one_hot = MultiLabelBinarizer() # encoder for the  tags 
y_onehot = one_hot.fit_transform(y.str.split(' ')) 
y_bin = pd.DataFrame(y_onehot, columns=one_hot.classes_ ) # transform it to Pandas object

In [ ]:
# tokenize
max_features = 5000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X))
list_tokenized_train = tokenizer.texts_to_sequences(X)
list_tokenized_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
#fix max comment lenght
maxlen = 100
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

## 4.The Model

In [ ]:
#initialize parameters
inp = Input(shape=(maxlen, )) #maxlen defined earlier
embed_size = 128

In [ ]:
# Neural network backbone
x = Embedding(max_features, embed_size)(inp)

x = LSTM(64, return_sequences=True,name='lstm_layer')(x)

x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(len(y_bin.columns), activation="softmax")(x)

In [ ]:
# build the model
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
# train the model
batch_size = 16
epochs = 3

hist = model.fit(X_t,y_onehot, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/3
2055/2055 [==============================] - 64s 31ms/step - loss: 0.2662 - accuracy: 0.3807 - val_loss: 0.2400 - val_accuracy: 0.4573
Epoch 2/3
2055/2055 [==============================] - 60s 29ms/step - loss: 0.2282 - accuracy: 0.4693 - val_loss: 0.2286 - val_accuracy: 0.4625
Epoch 3/3
2055/2055 [==============================] - 61s 30ms/step - loss: 0.2137 - accuracy: 0.5011 - val_loss: 0.2276 - val_accuracy: 0.4600


## 5.The prediction

In [ ]:
y_pred = model.predict(X_te, batch_size=batch_size, verbose=1)
print(y_pred.shape)

329/329 [==============================] - 2s 7ms/step
(5250, 19)


In [ ]:
df_probs_all = pd.DataFrame(y_pred,columns=y_bin.columns)

df_probs_all.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0.153458,0.193579,0.093476,0.058507,0.073517,0.005869,0.012124,0.058732,0.061029,0.000154,0.041969,0.009908,0.002006,0.008534,0.011478,0.154150,0.052170,0.005598,0.003740
1,0.223497,0.101688,0.009904,0.007503,0.040107,0.014141,0.073879,0.177815,0.009369,0.000302,0.024966,0.004687,0.000553,0.008947,0.005353,0.141232,0.138839,0.016359,0.000859
2,0.004208,0.004079,0.004338,0.001711,0.032020,0.002470,0.805717,0.111827,0.001297,0.000026,0.003374,0.000335,0.006187,0.002372,0.004732,0.004332,0.007658,0.003286,0.000032
3,0.072988,0.059212,0.032437,0.021755,0.063514,0.004167,0.001259,0.038086,0.066671,0.000185,0.325496,0.002237,0.000266,0.016921,0.009685,0.161812,0.122468,0.000339,0.000502
4,0.053394,0.007850,0.001079,0.001782,0.059151,0.238846,0.033053,0.297398,0.001552,0.017946,0.016155,0.000483,0.001904,0.059151,0.016429,0.003443,0.183221,0.004662,0.002502


In [ ]:
def top_5_predictions(df):
    N = 5
    cols = df.columns[:-1].tolist()
    a = df[cols].to_numpy().argsort()[:, :-N-1:-1]
    c = np.array(cols)[a]
    d = df[cols].to_numpy()[np.arange(a.shape[0])[:, None], a]
    df1 = pd.DataFrame(c).rename(columns=lambda x : f'max_{x+1}_col')

    predicted_genres = df1["max_1_col"] + ' ' + df1["max_2_col"]+ ' ' +df1["max_3_col"]+ ' ' + df1["max_4_col"]+ ' '+df1["max_5_col"]
    return predicted_genres

In [ ]:
pred_gen = top_5_predictions(df_probs_all)

In [ ]:
submission = pd.DataFrame(data= {'movie_id':test.movie_id,'predicted_genres':pred_gen})

In [ ]:
submission.head()

,movie_id,predicted_genres
0,10827,Adventure Sci-Fi Action Animation Comedy
1,51768,Action Drama Sci-Fi Thriller Adventure
2,36064,Documentary Drama Comedy Thriller Musical
3,33763,Horror Sci-Fi Thriller Action Fantasy
4,1146,Drama Crime Thriller Comedy Mystery


In [ ]:
submission.to_csv('submission.csv',index=False)